In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout, Input, Concatenate,concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import keras.backend as K

Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [3]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,dont worry ism girl,hmm how do i know if you are,whats ur name ? ?,others
1,1,when did in i?,saw many times i think -_-,no . i never saw you .,angry
2,2,by,by google chrome,where you live,others
3,3,u r ridiculous,i might be ridiculous but i am telling the tru...,u little disgusting whore,angry
4,4,just for time pass,wt do u do a a living then 4,maybe,others


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 3000
MAX_PROP_LENGTH = 64
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train_1 = tokenizer.texts_to_sequences(df["turn1"])
X_train_1 = pad_sequences(X_train_1, maxlen = MAX_PROP_LENGTH)

X_train_2 = tokenizer.texts_to_sequences(df["turn2"])
X_train_2 = pad_sequences(X_train_2, maxlen = MAX_PROP_LENGTH)

X_train_3 = tokenizer.texts_to_sequences(df["turn3"])
X_train_3 = pad_sequences(X_train_3, maxlen = MAX_PROP_LENGTH)

In [6]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train = []

for idx,row in df.iterrows():
    Y_train.append(one_hot_vector(row['label']))

Y_train = np.array(Y_train)

In [7]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,then dont ask me,youre a guy not as if you would understand,im not a guy fuck off,angry
1,1,mixed things such as ??,the things you do . .,have you seen minions ??,others
2,2,today ism very happy,and ism happy for you,i will be marry,happy
3,3,noah bring me some,left it there oops,orb,others
4,4,it is thoo,i said soon master . .,he is pressuring me,others


In [8]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [9]:
#tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')

#tokenizer.fit_on_texts(text_data)

X_test_1 = tokenizer.texts_to_sequences(df["turn1"])
X_test_1 = pad_sequences(X_test_1, maxlen = MAX_PROP_LENGTH)

X_test_2 = tokenizer.texts_to_sequences(df["turn2"])
X_test_2 = pad_sequences(X_test_2, maxlen = MAX_PROP_LENGTH)

X_test_3 = tokenizer.texts_to_sequences(df["turn3"])
X_test_3 = pad_sequences(X_test_3, maxlen = MAX_PROP_LENGTH)

In [10]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test = []

for idx,row in df.iterrows():
    Y_test.append(one_hot_vector(row['label']))

Y_test = np.array(Y_test)

In [11]:
embed_dim = 64
lstm_out = 32
batch_size = 64

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)

I_1 = Input(shape=X_train_1.shape[1:])
O_1 = Embedding(NR_WORDS, embed_dim,input_length = X_train_1.shape[1])(I_1)
O_1 = LSTM(lstm_out, dropout=0.5)(O_1)

I_2 = Input(shape=X_train_2.shape[1:])
O_2 = Embedding(NR_WORDS, embed_dim,input_length = X_train_2.shape[1])(I_2)
O_2 = LSTM(lstm_out, dropout=0.5)(O_2)

I_3 = Input(shape=X_train_3.shape[1:])
O_3 = Embedding(NR_WORDS, embed_dim,input_length = X_train_3.shape[1])(I_3)
O_3 = LSTM(lstm_out, dropout=0.5)(O_3)

result = concatenate([O_1,O_2,O_3])
result = Dense(4,activation='softmax')(result)
model = Model(inputs=[I_1,I_2,I_3], outputs=result,name="separate_embedings_model")
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])

In [12]:
model.build(np.array([X_train_1,X_train_2,X_train_3]).shape)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 64)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 64, 64)       192000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [13]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [14]:
Y_train = np.array(Y_train)
total = len(Y_train)
try:
    history = model.fit([X_train_1,X_train_2,X_train_3], Y_train,
                        validation_data=([X_test_1,X_test_2,X_test_3], Y_test),
                        epochs=10, verbose=1, batch_size=batch_size,
                        class_weight={
                            0: total / len(np.where(Y_train[:,0]==1.0)[0]),
                            1: total / len(np.where(Y_train[:,1]==1.0)[0]),
                            2: total / len(np.where(Y_train[:,2]==1.0)[0]),
                            3: total / len(np.where(Y_train[:,3]==1.0)[0]),
                        },callbacks=[mdcheck],shuffle=True)
except KeyboardInterrupt:
    pass

Train on 30160 samples, validate on 2755 samples
Epoch 1/20
30160/30160 [==============================] - 66s 2ms/step - loss: 2.5574 - acc: 0.7333 - f1: 0.6799 - val_loss: 0.6422 - val_acc: 0.7659 - val_f1: 0.6202
Epoch 2/20
30160/30160 [==============================] - 63s 2ms/step - loss: 1.7140 - acc: 0.8388 - f1: 0.8230 - val_loss: 0.5437 - val_acc: 0.8120 - val_f1: 0.6327
Epoch 3/20
30160/30160 [==============================] - 66s 2ms/step - loss: 1.5273 - acc: 0.8591 - f1: 0.8439 - val_loss: 0.5945 - val_acc: 0.7993 - val_f1: 0.6078
Epoch 4/20
30160/30160 [==============================] - 95s 3ms/step - loss: 1.3858 - acc: 0.8708 - f1: 0.8579 - val_loss: 0.4461 - val_acc: 0.8505 - val_f1: 0.6485
Epoch 5/20
30160/30160 [==============================] - 91s 3ms/step - loss: 1.3037 - acc: 0.8794 - f1: 0.8681 - val_loss: 0.5287 - val_acc: 0.8196 - val_f1: 0.6272
Epoch 6/20
30160/30160 [==============================] - 89s 3ms/step - loss: 1.2201 - acc: 0.8842 - f1: 0.8733 - v

In [15]:
save_model(model)

In [16]:
df_test = functions.parse_file(r"raw_data/EmoContext/devwithlabels2.txt", "EmoContext")
df_test.head()

,id,turn1,turn2,turn3,label
0,0,then dont ask me,youre a guy not as if you would understand,im not a guy fuck off,angry
1,1,mixed things such as ??,the things you do . .,have you seen minions ??,others
2,2,today ism very happy,and ism happy for you,i will be marry,happy
3,3,noah bring me some,left it there oops,orb,others
4,4,it is thoo,i said soon master . .,he is pressuring me,others


In [17]:
res = model.predict([X_test_1,X_test_2,X_test_3], batch_size=64, verbose=1)
res[3]

2755/2755 [==============================] - 2s 885us/step


array([3.2564381e-03, 9.2381379e-07, 8.0551277e-04, 9.9593711e-01],
      dtype=float32)

In [18]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [19]:
results = []
for r in res:
    results.append(softmax_convert(r))
    
df_test['label'] = results
df_test.head(50)
df_test.to_csv("separate_goodies.txt",index=False , sep="\t")